# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

def tokenize(text):
    sentence=str(text)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    rem_punct = re.sub(r"[^a-zA-Z0-9]", ' ', sentence)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', rem_punct)
    rem_url=re.sub(r'http\S+', ' ',cleantext)
    rem_num = re.sub('[0-9]+', ' ', rem_url)
    tokens = nltk.word_tokenize(text)
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stop_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w,pos='v') for w in lemma_words]
    #if stemmer:
    #  stem_words=[stemmer.stem(w) for w in lemma_words]

    return " ".join(lemma_words)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline,FeatureUnion
forest_clf = RandomForestClassifier(n_estimators=10)
pipeline = Pipeline([
                      
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('MultiOutput', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
from sklearn.model_selection import train_test_split
#Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#Train pipeline
pipeline.fit(X_train, Y_train)




Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
from sklearn.metrics import classification_report
Y_pred = pipeline.predict(X_test)

for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.47      0.28      0.35      2030
          1       0.80      0.91      0.85      6542
          2       0.50      0.01      0.03        68

avg / total       0.72      0.75      0.72      8640

request
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      7174
          1       0.63      0.16      0.25      1466

avg / total       0.81      0.84      0.80      8640

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8599
          1       0.00      0.00      0.00        41

avg / total       0.99      1.00      0.99      8640

aid_related
             precision    recall  f1-score   support

          0       0.65      0.82      0.73      5091
          1       0.60      0.38      0.46      3549

avg / total       0.63      0.64      0.62      8640

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import GridSearchCV

In [16]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'MultiOutput__estimator__n_estimators': [10, 20],
    'MultiOutput__estimator__min_samples_split': [2, 4],
    'MultiOutput__estimator__criterion': ("gini", "entropy")

}

cv = GridSearchCV(pipeline, parameters, cv=3, verbose=1,n_jobs=-1)
cv.fit(X_train, Y_train)
print("Best parameter (CV score=%0.3f):" % cv.best_score_)
print(cv.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.1min finished


Best parameter (CV score=0.186):
{'MultiOutput__estimator__n_estimators': 20}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
pipeline.set_params(**cv.best_params_)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [38]:
def evaluate_model(model, X_test, Y_test, category_names):
    """
    Args:
        model (pandas dataframe): the scikit-learn fitted model
        X_text (pandas dataframe): The X test set
        Y_test (pandas dataframe): the Y test classifications
        category_names (list): the category names
    Returns:
        Global Accuracy,Precision,Recall and classification_report for each labels
    """
    Y_pred = model.predict(X_test)
    print('Performance for each labels\n')
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(Y_test[col], Y_pred[:, i]))
evaluate_model(pipeline,X_test,Y_test,list(Y_train.columns))    

Performance for each labels

related
             precision    recall  f1-score   support

          0       0.55      0.23      0.32      2030
          1       0.79      0.94      0.86      6542
          2       0.67      0.03      0.06        68

avg / total       0.73      0.77      0.73      8640

request
             precision    recall  f1-score   support

          0       0.85      0.99      0.91      7174
          1       0.68      0.14      0.24      1466

avg / total       0.82      0.84      0.80      8640

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8599
          1       0.00      0.00      0.00        41

avg / total       0.99      1.00      0.99      8640

aid_related
             precision    recall  f1-score   support

          0       0.66      0.81      0.73      5091
          1       0.59      0.40      0.47      3549

avg / total       0.63      0.64      0.62      8640

medical_help
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [46]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator,TransformerMixin

In [47]:
class LengthTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [[len(x)] for x in X]

In [49]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
             ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
            ('SVD', TruncatedSVD(n_components=50)),
        ])),

        ('Length', LengthTransformer())
    ])),

    ('MultiOutput', MultiOutputClassifier(forest_clf))
])
pipeline.fit(X_train, Y_train)
evaluate_model(pipeline,X_test,Y_test,list(Y_train.columns))   

Performance for each labels

related
             precision    recall  f1-score   support

          0       0.46      0.30      0.36      2030
          1       0.80      0.90      0.84      6542
          2       0.00      0.00      0.00        68

avg / total       0.71      0.75      0.72      8640

request
             precision    recall  f1-score   support

          0       0.85      0.98      0.91      7174
          1       0.64      0.16      0.26      1466

avg / total       0.82      0.84      0.80      8640

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8599
          1       0.00      0.00      0.00        41

avg / total       0.99      1.00      0.99      8640

aid_related
             precision    recall  f1-score   support

          0       0.65      0.81      0.72      5091
          1       0.58      0.36      0.45      3549

avg / total       0.62      0.63      0.61      8640

medical_help
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [51]:
import joblib
joblib.dump(pipeline, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.